In [ ]:
from collections import defaultdict
import importlib
import joblib.parallel
from joblib import Parallel, delayed
import multiprocessing
import numpy as np
import os
import pandas as pd
from tqdm import tqdm
from time import time
from utils import get_spec

In [ ]:
keys_path = './keys.csv'
save_dir = './spectra/'

In [ ]:
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

In [ ]:
keys = pd.read_csv(keys_path)
print(len(keys))
keys.head()

In [ ]:
class BatchCompletionCallBack(object):
  completed = defaultdict(int)

  def __init__(self, time, index, parallel):
    self.index = index
    self.parallel = parallel

  def __call__(self, index):
    BatchCompletionCallBack.completed[self.parallel] += 1
    progr = BatchCompletionCallBack.completed[self.parallel]
    if progr in progr_index:
        perc = float(progr)/float(total)
        t = time()-init_time
        print("Progress: ","%0.2f"% (perc), "%, Time: ", "%0.2f"% t, "s.")
        progr_index.remove(progr)
    if self.parallel._original_iterator is not None:
        self.parallel.dispatch_next()

joblib.parallel.BatchCompletionCallBack = BatchCompletionCallBack

In [ ]:
total = len(keys)
progr_index = [total/10*i for i in range(10)]
progr_index = set(np.array(np.floor(progr_index), dtype=np.int32))

n_cores = multiprocessing.cpu_count()
init_time = time()
res = Parallel(n_jobs=8)(delayed(get_spec)(row, save_dir) for index,row in keys.iterrows())
e = time()
print("Total time: ", e-init_time, "s.")